---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 2 - Pandas Introduction
All questions are weighted the same in this assignment.
## Part 1
The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. 

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals. Use this dataset to answer the questions below.

In [2]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


### Question 0 (Example)

What is the first country in df?

*This function should return a Series.*

In [18]:
# You should write your whole answer within the function provided. The autograder will call
# this function and compare the return value against the correct solution value
def answer_zero():
    # This function returns the row for Afghanistan, which is a Series object. The assignment
    # question description will tell you the general format the autograder is expecting
    return df.iloc[0]

# You can examine what your function returns by calling it in the cell. If you have questions
# about the assignment formats, check out the discussion forums for any FAQs
answer_zero() 

# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
Name: Afghanistan, dtype: object

### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*

In [21]:
def answer_one(): 
    return df['Gold'].argmax() # argmax returns index of first occurrence of maximum of values

'United States'

### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*

In [39]:
def answer_two():
    return abs(df['Gold'] - df['Gold.1']).argmax()
answer_two()

'United States'

### Question 3
Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? 

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*

In [38]:
def answer_three():
    summer_gold = df.where((df['Gold'] > 0) & (df['Gold.1'] > 0))['Gold'] # countries with at least 1 gold in both summer and winter
    winter_gold = df.where((df['Gold'] > 0) & (df['Gold.1'] > 0))['Gold.1']
    return (abs(summer_gold - winter_gold) / df['Gold.2']).argmax()
answer_three()

'Bulgaria'

### Question 4
Write a function that creates a Series called "Points" which is a weighted value where each gold medal (`Gold.2`) counts for 3 points, silver medals (`Silver.2`) for 2 points, and bronze medals (`Bronze.2`) for 1 point. The function should return only the column (a Series object) which you created.

*This function should return a Series named `Points` of length 146*

In [53]:
def answer_four():
    Points = pd.Series(df['Gold.2'] * 3 + df['Silver.2'] * 2 + df['Bronze.2'] * 1)
    len(Points) # ensure length of series is 146
    return Points
answer_four()

Afghanistan                            2
Algeria                               27
Argentina                            130
Armenia                               16
Australasia                           22
Australia                            923
Austria                              569
Azerbaijan                            43
Bahamas                               24
Bahrain                                1
Barbados                               1
Belarus                              154
Belgium                              276
Bermuda                                1
Bohemia                                5
Botswana                               2
Brazil                               184
British West Indies                    2
Bulgaria                             411
Burundi                                3
Cameroon                              12
Canada                               846
Chile                                 24
China                               1120
Colombia        

## Part 2
For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov/popest/data/counties/totals/2015/CO-EST2015-alldata.html). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](http://www.census.gov/popest/data/counties/totals/2015/files/CO-EST2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.

### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

*This function should return a single string value.*

In [195]:
census_df = pd.read_csv('census.csv')
census_df[3170:]

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
3170,50,4,8,56,1,Wyoming,Albany County,36299,36299,36428,...,3.708956,2.637812,-3.544634,-3.334877,-9.911169,6.736119,6.433032,0.719587,1.429233,-5.166460
3171,50,4,8,56,3,Wyoming,Big Horn County,11668,11668,11672,...,4.868258,2.804930,16.815908,-8.026420,5.095861,4.868258,3.144921,17.236306,-7.608378,5.513554
3172,50,4,8,56,5,Wyoming,Campbell County,46133,46133,46244,...,-2.843479,15.601020,-5.895711,-8.550911,10.916963,-2.649606,15.558684,-5.916543,-8.509402,10.978525
3173,50,4,8,56,7,Wyoming,Carbon County,15885,15885,15837,...,-7.581980,-13.081441,3.178134,-2.970641,-23.300971,-7.392431,-12.636926,3.623073,-2.338590,-22.600668
3174,50,4,8,56,9,Wyoming,Converse County,13833,13833,13826,...,-12.847499,15.493820,19.035533,-20.550587,-0.070403,-12.774915,16.502720,20.093063,-19.358233,1.126443
3175,50,4,8,56,11,Wyoming,Crook County,7083,7083,7114,...,-1.544618,-4.202564,1.397819,6.378258,18.629317,-0.982939,-3.642222,2.096729,7.071547,19.309219
3176,50,4,8,56,13,Wyoming,Fremont County,40123,40123,40222,...,2.747083,7.782673,-4.990688,-12.331633,-13.673610,3.093562,8.027411,-4.747240,-12.013555,-13.352750
3177,50,4,8,56,15,Wyoming,Goshen County,13249,13247,13408,...,14.293649,3.961413,-8.079028,-7.017803,-11.899450,14.886132,4.841727,-6.903896,-5.761986,-10.635133
3178,50,4,8,56,17,Wyoming,Hot Springs County,4812,4812,4813,...,3.322604,6.208609,3.095336,-6.017222,-5.454164,5.191569,6.001656,2.888981,-6.224712,-5.663940
3179,50,4,8,56,19,Wyoming,Johnson County,8569,8569,8581,...,4.995063,-4.058912,-0.812583,-10.715742,0.933652,5.227392,-4.058912,-0.812583,-10.715742,0.933652


In [69]:
def answer_five():
    state_series = pd.Series(census_df['STNAME']) # create a series with the STNAME column
    # use value_counts() which returns an object containing counts of unique values 
    # this object is sorted in descending order, so take the first index
    return state_series.value_counts().index[0]
answer_five()

'Texas'

### Question 6
Only looking at the three most populous counties for each state, what are the three most populous states (in order of highest population to lowest population)? Use `CENSUS2010POP`.

*This function should return a list of string values.*

In [29]:
def answer_six():
    census_df = pd.read_csv('census.csv')
    census_df = census_df[census_df['SUMLEV'] == 50]
    columns_to_keep = ['STNAME', 'CTYNAME', 'POPESTIMATE2010',]
    census_df = census_df[columns_to_keep]
    #census_df = census_df.set_index(['STNAME', 'CTYNAME'])
    pop_state = census_df.groupby('STNAME')['POPESTIMATE2010'].apply(lambda grp: grp.nlargest(3).sum()) #http://stackoverflow.com/questions/40390634/pandas-groupby-nlargest-sum
    return list(pop_state.nlargest(3).index)
answer_six()




['California', 'Texas', 'Illinois']

### Question 7
Which county has had the largest absolute change in population within the period 2010-2015? (Hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all six columns.)

e.g. If County Population in the 5 year period is 100, 120, 80, 105, 100, 130, then its largest change in the period would be |130-80| = 50.

*This function should return a single string value.*

In [66]:
def answer_seven():
    census_df = pd.read_csv('census.csv')
    census_df = census_df[census_df['SUMLEV'] == 50] # filter at summary level
    columns_to_keep = ['CTYNAME', # filter only the needed columns
                   'POPESTIMATE2010',
                   'POPESTIMATE2011',
                   'POPESTIMATE2012',
                   'POPESTIMATE2013',
                   'POPESTIMATE2014',
                   'POPESTIMATE2015']
    census_df = census_df[columns_to_keep]
    census_df = census_df.set_index('CTYNAME') # set index to CTYNAME
    census_df.sort_index(inplace=True)
    # calculate population change for all 6 columns
    census_df['POPCHANGE'] = census_df.iloc[:, 0:6].max(axis=1, numeric_only=True)- census_df.iloc[:, 0:6].min(axis=1, numeric_only=True)
    return census_df['POPCHANGE'].argmax() # find the largest population, and return its index
answer_seven()

'Harris County'

### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [93]:
def answer_eight():
    census_df = pd.read_csv('census.csv')
    census_df = census_df[((census_df['REGION'] == 1) | (census_df['REGION'] == 2))
    & (census_df['CTYNAME'].str.startswith('Washington'))
    & (census_df['POPESTIMATE2015'] > census_df['POPESTIMATE2014'])].iloc[:, [5,6]]
    return census_df
answer_eight()

,STNAME,CTYNAME
896,Iowa,Washington County
1419,Minnesota,Washington County
2345,Pennsylvania,Washington County
2355,Rhode Island,Washington County
3163,Wisconsin,Washington County
